# Static GCP calibration

Notebook which computes and saves results of the static GCP calibration

In [ ]:
import numpy as np
import importlib
import matplotlib.pyplot as plt
import scripts.theodolite_utils as theodo_u
import scripts.theodolite_plot_function as theodo_p
import scripts.theodolite_function as theodo_f
import scripts.resection_functions as theodo_r

In [ ]:
# This cell gathers result for the static control points for the graphic below

theodo_r = importlib.reload(theodo_r)
theodo_u = importlib.reload(theodo_u)
theodo_f = importlib.reload(theodo_f)

path_list = [
    "../data/20221129-5/",
]

path_list_inter = [
    ["../data/20221129-5/"],
]

save_results = True

errors_static_cp = []
error_inter_prism_exp_cp = []
TF_list_static = []
for i,j in zip(path_list,path_list_inter):
    # Generate file path
    file_GCP = i + 'total_stations/GCP.txt'
    print("don't give up: ", i)
    error_static_loop = []
    error_static_exp_loop = []
    error_static_cp_loop = []
    Tf_static_loop = []
    for m in j:
        file_prediction = theodo_u.if_file_exist(m + 'filtered_prediction/f-2-2-2-4-4-0-L_','1.csv')
        file_sensors = theodo_u.if_file_exist(m + "sensors_extrinsic_calibration/calibration_results.csv",'')
        extrinsic_calibration_results = theodo_u.read_extrinsic_calibration_results_file(file_sensors)
        errors_cp, error_inter_prism_cp, TF_list = theodo_r.static_control_points_error(static_file_path=file_GCP, exp_file_path=file_prediction, inter_prism_dist=extrinsic_calibration_results)
        error_static_exp_loop += error_inter_prism_cp
        error_static_cp_loop += errors_cp
        Tf_static_loop += TF_list
    errors_static_cp += error_static_cp_loop
    error_inter_prism_exp_cp += error_static_exp_loop
    TF_list_static += Tf_static_loop
    print("Cp error: ",np.median(error_static_cp_loop),np.std(error_static_cp_loop))
    print("Inter-prism: ", np.median(error_static_exp_loop),np.std(error_static_exp_loop))
    if(save_results):
        theodo_u.save_error_list_to_file(error_static_cp_loop, i+"list_error/error_static_cp_cp.csv")
        theodo_u.save_error_list_to_file(error_static_exp_loop, i+"list_error/error_static_cp_experiment.csv")
        theodo_u.save_tf_list_to_file_multi(Tf_static_loop, i+"list_tf/TF_list_static_cp.csv")
print("End !")


In [ ]:
# Display results according to different metrics
%matplotlib notebook

fig = plt.figure(figsize =(10, 5))
ax = fig.add_subplot(111)
lis = [errors_static_cp,error_inter_prism_exp_cp]

box = ax.boxplot(lis, notch=True, patch_artist=True, vert = 1, showfliers=False, showmeans=False)
plt.xticks([1,2],
               ["GCP\n\n\n M: "+str(round(np.median(lis[0]),2))+"mm \n Std: "+str(round(np.std(lis[0]),2))+"mm\n"+str(round(len(lis[0])/3))+" pts",
            "Inter-prism\n\n\n M: "+str(round(np.median(lis[1]),2))+"mm \n Std: "+str(round(np.std(lis[1]),2))+"mm\n"+str(round(len(lis[1])/3))+" pts",
           ])
colors_box = ['#069AF3','#069AF3']

for patch, color in zip(box['boxes'], colors_box):
    patch.set_facecolor(color)

ax.set_ylabel("Error [mm]")
fig.tight_layout()
plt.show()